In [1]:
import yfinance as yf

In [2]:
for ticker in ['GOOG', 'AAPL', 'META', 'TSLA', 'QQQ']:
    ticker_yahoo = yf.Ticker(ticker)
    history = ticker_yahoo.history(period = "2d")
    current = history.iloc[-1]['Close']
    previous = history.iloc[-2]['Close']
    print(f"Price of {ticker}, previous: {previous}, current {current}, change: {(current / previous - 1) * 100}%")

Price of GOOG, previous: 199.6300048828125, current 201.5803985595703, change: 0.9770042724302597%


Price of AAPL, previous: 222.63999938964844, current 222.74000549316406, change: 0.04491830030084465%
Price of META, previous: 616.4600219726562, current 623.364990234375, change: 1.1200999279114665%
Price of TSLA, previous: 424.07000732421875, current 417.8399963378906, change: -1.4690996483429752%


Price of QQQ, previous: 524.7999877929688, current 532.0700073242188, change: 1.385293388024622%


In [3]:
import openmeteo_requests
import requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 37.3394,
	"longitude": -121.895,
	"current": ["temperature_2m", "relative_humidity_2m"],
	"daily": "weather_code",
	"timezone": "America/Los_Angeles"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print("Weather for San Jose")
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


# Current values. The order of variables needs to be the same as requested.
current = response.Current()

current_temperature_2m = current.Variables(0).Value()

current_relative_humidity_2m = current.Variables(1).Value()

print(f"Current time {current.Time()}")

print(f"Current temperature {current_temperature_2m}")
print(f"Current relative_humidity {current_relative_humidity_2m}")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

weather_code_desc = requests.get("https://gist.githubusercontent.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c/raw/76b0cb0ef0bfd8a2ec988aa54e30ecd1b483495d/descriptions.json").json()
daily_data["weather_code"] = [weather_code_desc[str(round(code))]['day']['description'] for code in daily_weather_code]
daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

Weather for San Jose
Coordinates 37.3475456237793°N -121.8808364868164°E
Elevation 33.0 m asl
Timezone b'America/Los_Angeles' b'GMT-8'
Timezone difference to GMT+0 -28800 s
Current time 1737557100
Current temperature 1.2284998893737793
Current relative_humidity 63.0


                       date weather_code
0 2025-01-22 08:00:00+00:00        Sunny
1 2025-01-23 08:00:00+00:00        Sunny
2 2025-01-24 08:00:00+00:00       Cloudy
3 2025-01-25 08:00:00+00:00       Cloudy
4 2025-01-26 08:00:00+00:00       Cloudy
5 2025-01-27 08:00:00+00:00        Sunny
6 2025-01-28 08:00:00+00:00        Sunny
